## Utils

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

In [ ]:
def binomail_hyp_test(p0, n, n_success, sig_level=None):
    
    sig_level = 0.05 if not sig_level else sig_level
    
    sample_mean = n_success/n
    population_variance = p0*(1-p0)/n
    population_sd = np.sqrt(population_variance)
    
    print('Initial results ---------------------------------------------------------')
    print('\tLet X_i be a Bernoulli with p that indicates whether an event happens or not\n')
    print('\tLet X = X_1 + ··· + X_n. X follows a Binom(n, p) which Var = n*p*(1-p)\n')
    print('\tLet p_hat = X/n. p_hat follows a Binom(n/p, p) which Var = p*(1-p)/n\n')
    print('\tWhen n is large, p_hat can be approximated by a Norm(p, p*(1-p)/n)\n')
    
    print('Hypothesis test ---------------------------------------------------------')
    print('\tH0: p={} \n\t  vs\n\tH1: p<{}'.format(p0, p0))
    
    print('\nStatistics --------------------------------------------------------------')
    print('\tSample Mean: {}'.format(sample_mean))
    print('\tp0: {}'.format(p0))
    
    print('\nPlots -------------------------------------------------------------------')
    q = stats.norm.ppf(sig_level, 0, 1)
    
    x = np.linspace(p0 - 3*population_sd, p0 + 3*population_sd, 1000)
    y = stats.norm.pdf(x, p0, population_sd)

    print('\t Under H0, p_hat follows a Norm({}, {})'.format(p0, population_variance))
    plt.plot(x, y)
    plt.title('Probability distribution for p_hat under H0')
    
    plt.axvline(x=sample_mean, color='k', linestyle='--')
    plt.text(sample_mean+0.05*np.abs(sample_mean), np.mean(y), 'Sample Mean', rotation=90)
    
    plt.axvline(x=p0, color='k', linestyle='--')
    plt.text(p0+0.05*np.abs(p0), np.mean(y), 'p0', rotation=90)
    
    plt.show()
    
    print('\tGiven the sample, the value of p_hat is: {:.5}'.format(sample_mean))
    print('\tsample mean - p0 = {:.5}'.format(sample_mean - p0))
    
    print('\nDiscrepancy statistic -----------------------------------------------------')
    print('\td = (p_hat - p0)/sqrt(p0 * (1 - p0)/n)')
    print('\tUnder H0, d follows a N(0,1)')
    
    t_n = (sample_mean - p0)/np.sqrt(p0*(1-p0)/n)
    p_val = stats.norm.cdf(t_n, 0, 1)
    
    x = np.linspace(0 - 3*1, 0 + 3*1, 1000)
    y = stats.norm.pdf(x, 0, 1)
    
    plt.plot(x, y)
    plt.title('Probability distribution for d under H0')
    
    plt.axvline(x=t_n, color='k', linestyle='--')
    plt.text(t_n+0.05*np.abs(t_n), np.mean(y), 't_n for the sample', rotation=90)
    
    plt.axvline(x=q, color='k', linestyle='--')
    plt.text(q+0.05*np.abs(q), np.mean(y), 'Upper limit rej. area', rotation=90)
    
    plt.show()
    
    
    print('\tValue of d for the given sample: d_sample = {}'.format(t_n))
    print('\tRejection area (in terms of d) = {' + 'values for d such that d<= {:.4f}'.format(q) + '}')
    print('\tAccepted area  (in terms of d) = {' + 'values for d such that d> {:.4f}'.format(q) + '}')
    print('\tp-value = Prob(d|H0 <= d_sample) =  {:.4f}'.format(p_val))
    print("\n")
    print('\tRejection area (in terms of p_hat):') 
    print('\t{(x_1, x_2, ..., x_n) such that ' + 'p_hat = (x_1 + x_2 + ··· + x_n)/n<= {:.4f}'.format(q*population_sd+p0) + '}')
    

In [ ]:
def power_function_report(p0, p, n, sig_level=None):
    sig_level = 0.05 if not sig_level else sig_level
    
    def calculate_power(p):
        q = stats.norm.ppf(sig_level, 0, 1)
        num = q * np.sqrt(p0*(1-p0)/n) + p0 - p
        denum = np.sqrt(p*(1-p)/n)
        pf_p0_p = stats.norm.cdf(num/denum)
        return pf_p0_p
    
    def generate_power_function():
        p_ini = p0 - 0.7*p0 if p0 - 0.7*p0 >=0 else 0
        p_end = p0 + 0.1*p0 if p0 - 0.1*p0 <= 1 else 1
        
        steps = 10**4
        h = (p_end - p_ini)/steps
        probs = []
        pf = []
        i = 0
        
        while(i<steps):
            prob_i = p_ini + i*h
            probs.append(prob_i)
            pf.append(calculate_power(p=prob_i))
            i = i + 1
            
        return probs, pf
        
    sample_power = calculate_power(p=p)
    
    print('Initial results ---------------------------------------------------------')
    print('\tPF = Power function = Prob(reject H0 | p)\n')
    print('\tWhen p belongs to the rejection area we have the following result:\n') 
    print('\tPF = Prob(reject H0 | H0 is false) =')
    print('\t1 - Prob(accept H0 | H0 is false) =')
    print('\t1 - Prob(error type II)\n')
    
    print('Hypothesis test ---------------------------------------------------------')
    print('\tH0: p={} \n\t  vs\n\tH1: p<{}\n'.format(p0, p0))
    
    
    print('Plot --------------------------------------------------------------------')
    probs, pf = generate_power_function()
    plt.axvline(x=p0, color='k', linestyle='--')
    plt.text(p0+0.0001*np.abs(p0), np.mean(pf), 'p0', rotation=90)
    
    pf_p0 = calculate_power(p=p0)
    plt.axhline(y=pf_p0, color='k', linestyle='--')
    plt.text(np.mean(probs), pf_p0+0.3*pf_p0, round(pf_p0, 4))
    
    plt.plot(probs, pf)
    plt.show()
    
    print('Stats --------------------------------------------------------------------')
    print('\tGiven p={}, the associated power is: {}'.format(p, sample_power))

## Assumtions
- The development of this notebook assumes that the significance level, $\alpha$ is 0.05:
$$\alpha = 0.05 = Prob(\text{reject } H_0 | H_0 \text{ is true}) = Prob(\text{error type I}) = $$
$$ Prob(\text{see an effect when actually there is no effect})$$

## First situation

The company we work for produces digital cameras. The error made during the process of manufacturing is $p_0$=0.01. It means that out of 100 cameras, 1 is faulty. 

In order to lower the error we review the whole process. Some "weak" points are detected and fixed. To see if the error is lower than 0.001, we made the following experiment: 5000 cameras are procuded and inspected. 3 out of these 5000 are faulty, which means that 3/5000 = 0.0006, which is lower than 0.001. Is the new error rate $p$ (which is unknown) lower than the original one ($p_0$=0.01)?

In order to anser this question we perform and hypothesis test:

 \begin{cases} 
     H_0 & p= 0.01 \\
     H_1 & p\lt 0.01
\end{cases}

We want a test with a 95% significance level.

In [ ]:
binomail_hyp_test(p0=0.001, n=5000, n_success=3, sig_level=0.05)

## Second situation

The company we work for produces digital cameras. The error made during the process of manufacturing is $p_0$=0.02. It means that out of 100 cameras, 2 are faulty. 

In order to lower the error we review the whole process. Some "weak" points are detected and fixed. To see if the error is lower than 0.002, we made the following experiment: 5000 cameras are procuded and inspected. 3 out of these 5000 are faulty, which means that 3/5000 = 0.0006, which is lower than 0.002. Is the new error rate $p$ (which is unknown) lower than the original one ($p_0$=0.02)?

In order to anser this question we perform and hypothesis test:

 \begin{cases} 
     H_0 & p= 0.02 \\
     H_1 & p\lt 0.02
\end{cases}

We want a test with a 95% significance level.

In [ ]:
binomail_hyp_test(p0=0.002, n=5000, n_success=3)

In [ ]:
power_function_report(p0=0.002, p=3/5000, n=5000, sig_level=0.05)